In [1]:
# 四半期の騰落率が大きく動いた銘柄の過去分析行う
# 　四半期騰落率過去分析1: すでにある程度業績の良い銘柄の分析
# 　四半期騰落率過去分析2: 黒字化銘柄の分析

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [3]:
#####
##### 評価日における四半期騰落率一覧
#####

In [4]:
valuation_date = date(2024, 8, 1)

KPl = KessanPl()
updown_rate_df = KPl.get_quater_settlement_price_updown_rate(valuation_date, "nh225")
updown_rate_df

code,start_date,end_date,updown_rate,nh_updown_rate
i64,date,date,f64,f64
1301,2024-05-11,2024-08-05,-8.97,-17.67
1332,2024-05-15,2024-08-06,-12.67,-10.01
1333,2024-05-08,2024-08-05,-11.23,-18.67
1375,2024-05-10,2024-08-08,-0.41,-9.2
1376,2024-07-11,2024-10-04,-3.01,-8.76
…,…,…,…,…
9993,2024-07-12,2024-10-10,-5.32,-5.49
9994,2024-05-16,2024-08-09,-3.61,-9.37
9995,2024-05-08,2024-08-01,0.27,-1.43


In [5]:
#####
##### valuation_date時点で発表済四半期決算の業績成長率一覧(growth_rate_df)の作成
#####

In [6]:
KPL1 = KessanPl()
KPL2 = KessanPl()
KPL1.with_columns_growthrate_lastyear()
KPL2.with_columns_diff_growthrate()
KPL1.df = KPL1.df.filter(pl.col("settlement_type")=="四")
KPL2.df = KPL2.df.filter(pl.col("settlement_type")=="四")
df = KPL1.df.join(KPL2.df, on=["code", "settlement_date"], how="left")
KPL3 = KessanPl(df)
df =KPL3.get_latest_quater_settlements(valuation_date)
df = df.select([
    "code", "settlement_date", "announcement_date", "sales", "operating_income", "sales_growthrate", "diff_operating_income_growthrate"
])
df = df.rename({"settlement_date": "評価日発表済最新四半期決算"})
growth_rate_df = df
growth_rate_df

code,評価日発表済最新四半期決算,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,date,i64,i64,f64,f64
1301,2024-03-31,2024-05-10,59990,1857,2.1,155.0
1332,2024-03-31,2024-05-14,205894,3308,8.5,13.6
1333,2024-03-31,2024-05-07,243328,1176,1.6,-37.3
1375,2024-03-31,2024-05-09,11807,-803,21.1,-5.0
1376,2024-05-31,2024-07-10,23224,1150,1.9,-16.0
…,…,…,…,…,…,…
9993,2024-05-31,2024-07-11,24969,-408,1.8,-108.5
9994,2024-03-31,2024-05-15,36543,1082,3.1,63.8
9995,2024-03-31,2024-05-07,18708,63,7.7,0.1


In [7]:
#####
##### 評価dfの作成：　updown_rate_dfとgrowth_rate_dfをjoinして必要な列を残す
#####

In [12]:
df = growth_rate_df.join(updown_rate_df, on=["code"], how="left")
df = df.with_columns([
    (pl.lit(100) * pl.col("operating_income") / pl.col("sales")).round(2).alias("営業利益率")
])
df = df.select([
    "code",
    "sales",
    "operating_income",
    "営業利益率",
    "sales_growthrate",
    "diff_operating_income_growthrate",
    "start_date",
    "end_date",
    "updown_rate",
    "nh_updown_rate"	
])
evaluated_df = df
evaluated_df

code,sales,operating_income,営業利益率,sales_growthrate,diff_operating_income_growthrate,start_date,end_date,updown_rate,nh_updown_rate
i64,i64,i64,f64,f64,f64,date,date,f64,f64
1301,59990,1857,3.1,2.1,155.0,2024-05-11,2024-08-05,-8.97,-17.67
1332,205894,3308,1.61,8.5,13.6,2024-05-15,2024-08-06,-12.67,-10.01
1333,243328,1176,0.48,1.6,-37.3,2024-05-08,2024-08-05,-11.23,-18.67
1375,11807,-803,-6.8,21.1,-5.0,2024-05-10,2024-08-08,-0.41,-9.2
1376,23224,1150,4.95,1.9,-16.0,2024-07-11,2024-10-04,-3.01,-8.76
…,…,…,…,…,…,…,…,…,…
9993,24969,-408,-1.63,1.8,-108.5,2024-07-12,2024-10-10,-5.32,-5.49
9994,36543,1082,2.96,3.1,63.8,2024-05-16,2024-08-09,-3.61,-9.37
9995,18708,63,0.34,7.7,0.1,2024-05-08,2024-08-01,0.27,-1.43


In [9]:
#####
##### filter
#####

In [24]:
# 列フィルタパラメータ
min_sales = 0
min_operating_income = 0
min_営業利益率 = 0
min_sales_growthrate = 10

r1 = 0.8
r2 = 1.5

# フィルタ
df = evaluated_df
# 列フィルタ
df = df.filter(pl.col("sales") >= min_sales)\
    .filter(pl.col("operating_income") >= min_operating_income)\
    .filter(pl.col("営業利益率") >= min_営業利益率)\
    .filter(pl.col("sales_growthrate") >= min_sales_growthrate)
# 列間フィルタ
df = df.filter((pl.lit(r1) * pl.col("営業利益率")) <= pl.col("sales_growthrate"))\
    .filter((pl.lit(r2) * pl.col("sales_growthrate")) <= pl.col("diff_operating_income_growthrate")) 
target_df = df
target_df

code,sales,operating_income,営業利益率,sales_growthrate,diff_operating_income_growthrate,start_date,end_date,updown_rate,nh_updown_rate
i64,i64,i64,f64,f64,f64,date,date,f64,f64
1419,95225,11127,11.68,15.8,42.8,2024-07-12,2024-10-11,-7.47,-4.95
1435,2758,599,21.72,27.0,62.0,2024-05-14,2024-08-09,-14.29,-8.52
1438,1299,172,13.24,11.9,37.7,2024-05-11,2024-08-09,-16.91,-8.34
1805,38093,2286,6.0,11.1,31.9,2024-05-16,2024-08-09,-7.99,-9.37
1820,106790,3950,3.7,12.8,27.1,2024-05-14,2024-08-06,-0.78,-9.44
…,…,…,…,…,…,…,…,…,…
9755,18211,2017,11.08,16.3,28.8,2024-05-15,2024-08-09,-15.83,-9.1
9766,90039,25145,27.93,24.0,45.9,2024-05-10,2024-08-01,1.69,-0.61
9853,1884,6,0.32,11.1,36.2,2024-05-11,2024-08-08,-2.74,-8.85


In [25]:
#####
##### 統計結果出力
#####

In [26]:
df = evaluated_df
print("##### 全体")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'全銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')
print()

df = target_df
print("##### 抽出銘柄")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'抽出銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')

##### 全体
総銘柄数：3746
上昇銘柄数:921／下落銘柄数:2825／上昇率:(24.6%)
全銘柄平均騰落率:-6.34%
日経225平均騰落率:-6.14%

##### 抽出銘柄
総銘柄数：262
上昇銘柄数:76／下落銘柄数:186／上昇率:(29.0%)
抽出銘柄平均騰落率:-6.89%
日経225平均騰落率:-6.26%


In [27]:
#####
##### 個別銘柄の分析
#####

In [29]:
# 四季報
code = 1419
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

1419(タマホーム)の四季報データ履歴

発行日: 2024年06月14日
【上向く】
  25年5月期は主力の注文住宅販売が引き渡し棟数軟調続く。ただ、価格転嫁浸透して粗利率改善。強化中の分譲住宅販売やリフォーム積み増す。積極投入する販促費増こなし営業益上向く。増配余地。
【分譲住宅】
  事業拡大に向け、全国11カ所の土地買取センターで人員増強や地場不動産会社との関係性構築を進める。プリカなど進呈する展示場集客策続けて需要喚起。

発行日: 2024年03月16日
【一転減益】
  分譲住宅販売棟数は前期比1割増の1400戸想定。ただ、柱の注文住宅の販売棟数減少が想定超。後半の販促強化でも補えず。前号から一転営業減益。25年5月期は注文住宅持ち直し利益上向く。
【人員増】
  リフォーム事業の人員増など体制強化。定期点検などで顧客との関係構築、提案活動推進して事業成長図る。分譲住宅は専門ノウハウ持つ人材採用を進める。

発行日: 2023年12月15日
【連続増益】
  柱の注文住宅は序盤から販売棟数減も、販促強化で後半挽回。値上げ浸透し採算改善。分譲住宅販売棟数は前期比1割増の1400戸前提。リフォームやオフィス区分所有権販売堅調。連続営業増益。
【喚起策】
  23年10～11月に展示場来場者へプリペイドカード進呈キャンペーン実施。潜在客と接点増で低調な需要喚起。分譲は地方中核都市軸に小規模用地仕入れ強化。

発行日: 2023年09月15日
【連続増配】
  柱の注文住宅が販売棟数減ながら値上げ浸透して利益率向上。仕入れ強化中の分譲住宅は販売棟数が前期比1割超増の1400戸公算。オフィス区分所有権販売順調。販促費増こなして連続最高純益。
【重点】
  分譲住宅は資金回転率重視し10区画以下の小規模分譲地中心に仕入れ拡大、首都圏や関西軸に深耕。営業拠点は関東や東海の空白エリアへの出店を積極化。


In [30]:
#####
##### 四半期決算推移
#####

In [31]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [32]:
#####
##### 株価推移
#####

In [33]:
# 日足
delta = relativedelta(days=100)

KPL = KessanPl()
df = KPL.df.filter(pl.col("announcement_date")<valuation_date).filter(pl.col("code")==code)
ann_date = df["announcement_date"].to_list()[-1]
print(f'決算発表日: {ann_date}')
start_date = ann_date - delta
end_date = ann_date + delta

PFIG = PricelistFig(code, RevPl.df, MPL.df, start_date, end_date)
# 決算発表日にvlineを引く
PFIG.add_vline_announcement_date()
PFIG.fig.show()

決算発表日: 2024-07-11


In [34]:
#####
##### appendix
#####

In [43]:
KPL = KessanPl()
df = KPL.df

df = df.filter(pl.col("code")==code)\
    .filter(pl.col("announcement_date")<=valuation_date)\
    .filter(pl.col("settlement_type").is_in(["予","本"]))
df

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
1419,2017-05-31,"""本""",2017-07-14,157001,3901,3475,901,30.0,15.0,-2
1419,2018-05-31,"""本""",2018-07-13,167915,4653,4029,2047,68.1,30.0,4
1419,2019-05-31,"""本""",2019-07-10,186874,7366,6955,3934,130.9,53.0,4
1419,2020-05-31,"""本""",2020-07-13,209207,9873,9754,5109,172.8,70.0,4
1419,2021-05-31,"""予""",2020-07-13,195000,7500,7200,4000,135.8,60.0,4
…,…,…,…,…,…,…,…,…,…,…
1419,2023-05-31,"""本""",2023-07-11,256065,13264,13477,8715,298.4,180.0,4
1419,2024-05-31,"""予""",2023-07-11,257000,14100,14100,9200,317.4,185.0,4
1419,2024-05-31,"""予""",2024-06-18,247700,12500,12600,8800,303.6,185.0,4
